In [1]:
import os
import datetime
import numpy as np
import pandas as pd

import QuantLib as ql

In [2]:
from quant_lib.cds_curve import get_irs_quote, get_cds_quote, swap_curve, cds_curve

In [3]:
class CDS():
    def __init__(self, today, maturity_date, spread, recovery, notional, position):
        
        # initial setup
        self.date = today
        self.discount_curve_t0 = self.discount_curve(self.date)
        self.cds_curve_t0 = self.cds_curve(self.date)

        self.maturity_date = ql.Date(maturity_date.day, maturity_date.month, maturity_date.year)

        if position == 'long':
            self.position = ql.Protection.Buyer
        else:
            self.position = ql.Protection.Seller

        self.spread = spread
        self.notional = notional
        self.recovery_rate = recovery

        self.tenor = ql.Period(3, ql.Months)
        self.calendar =  ql.UnitedStates()
        self.convention = ql.ModifiedFollowing
        self.dateGeneration = ql.DateGeneration.CDS
        self.dayCount = ql.Actual360()
        self.endOfMonth = False

        # pricing result
        self.npv = self.pricing(self.discount_curve_t0, self.cds_curve_t0)
        self.ir_delta = self.ir_delta()
        self.credit_delta = self.credit_delta()
        self.theta = self.theta()

    def discount_curve(self, date):
        return swap_curve(date, get_irs_quote(date))
    
    def cds_curve(self, date):
        return cds_curve(date, get_cds_quote(date), swap_curve(date, get_irs_quote(date)))
    
    def pricing(self, discount_curve, cds_curve):
        # processing
        todays_date = ql.Date(self.date.day, self.date.month, self.date.year)
        discount_curve_handle = ql.YieldTermStructureHandle(discount_curve)

        schedule = ql.Schedule(todays_date,
                                self.maturity_date,
                                self.tenor,
                                self.calendar,
                                self.convention,
                                self.convention,
                                self.dateGeneration,
                                self.endOfMonth
                            )

        cds = ql.CreditDefaultSwap(self.position,
                                   self.notional,
                                   self.spread/1000,
                                   schedule,
                                   self.convention,
                                   self.dayCount
                                    )
        
        probability = ql.DefaultProbabilityTermStructureHandle(cds_curve)

        engine = ql.MidPointCdsEngine(probability=probability,
                                      recoveryRate=self.recovery_rate,
                                      discountCurve=discount_curve_handle
                                    )
        
        cds.setPricingEngine(engine)
        
        npv = cds.NPV()

        return npv
    
    def ir_delta(self):
        curve_handle = ql.YieldTermStructureHandle(self.discount_curve_t0)

        # 1bp
        basis_point = 0.0001

        # CDS price when 1bp up
        up_curve = ql.ZeroSpreadedTermStructure(curve_handle, ql.QuoteHandle(ql.SimpleQuote(basis_point)))
        up_cds = self.pricing(up_curve, self.cds_curve_t0)

        # CDS price when 1bp down
        down_curve = ql.ZeroSpreadedTermStructure(curve_handle, ql.QuoteHandle(ql.SimpleQuote(-basis_point)))
        down_cds = self.pricing(down_curve, self.cds_curve_t0)

        # interest rate delta
        return (up_cds - down_cds) / 2
    
    def credit_delta(self):
        _cds_quote = get_cds_quote(self.date)

        # CDS price when 1bp up
        _cds_quote['Market.Mid'] += 1
        up_curve = cds_curve(self.date, _cds_quote, self.discount_curve_t0)
        up_cds = self.pricing(self.discount_curve_t0, up_curve)

        # CDS price when 1bp down
        _cds_quote['Market.Mid'] -= 1
        down_curve = cds_curve(self.date, _cds_quote, self.discount_curve_t0)
        down_cds = self.pricing(self.discount_curve_t0, down_curve)

        # credit delta
        return (up_cds - down_cds) / 2

    def theta(self):
        price_t0 = self.pricing(self.discount_curve_t0, self.cds_curve_t0)

        discount_curve_t1 = self.discount_curve(self.date + datetime.timedelta(days=1))
        cds_curve_t1 = self.cds_curve(self.date + datetime.timedelta(days=1))
        price_t1 = self.pricing(discount_curve=discount_curve_t1, cds_curve=cds_curve_t1)

        theta = price_t1 - price_t0

        return theta


In [4]:
todays_date = datetime.date(2020, 12, 11)
maturity_date = datetime.date(2025, 12, 11)

notional = 10000000
spread = 20.5241
recovery = 0.4
position = 'short'


In [5]:
# build CCS object
cds = CDS(
        today=todays_date,
        maturity_date=maturity_date,
        spread=spread,
        recovery=0.4,
        notional=notional,
        position=position
        )

In [6]:
print("price of CDS = {}".format(round(cds.npv,4)))
print("IR Delta = {}".format(round(cds.ir_delta,4)))


print("Credit Delta = {}".format(round(cds.credit_delta,4)))
print("Theta = {}".format(round(cds.theta,4)))

price of CDS = 964810.2233
IR Delta = -236.2413
Credit Delta = -2693.3063
Theta = 75.4122
